In [1]:
from ETGEMs_function_PMI import *
import pandas as pd
import cobra
import ast
from cobra.io import write_sbml_model
from numpy import *
import copy

In [2]:
#Get Reaction G0 from local file_original
reaction_g0_file_original0 = './reaction_g0_strain0.txt'
reaction_g0_file_original1 = './reaction_g0_strain1.txt'
#Get Metabolite concentration from local file
metabolites_lnC_file_original0 = './metabolites_lnC_strain0.txt'
metabolites_lnC_file_original1 = './metabolites_lnC_strain1.txt'
#Get Model from local file
model_file_original0 = './iML1515.xml'
model_file_original1 = './iMM904.xml'
#Get reaction kcat data from ECMpy
reaction_kcat_MW_file_original0 = './ID_kcat_MW_file_strain0.csv'
reaction_kcat_MW_file_original1 = './ID_kcat_MW_file_strain1.csv'

In [3]:
## Convert to usable model formats
model0=Get_Concretemodel_Need_Data(reaction_g0_file_original0,metabolites_lnC_file_original0,model_file_original0,reaction_kcat_MW_file_original0)
model1=Get_Concretemodel_Need_Data(reaction_g0_file_original1,metabolites_lnC_file_original1,model_file_original1,reaction_kcat_MW_file_original1)
## knockout the fhuA gene
model0['lb_list']['EX_fecrm_e ']=0
model0['lb_list']['EX_fecrm_e ']=0

In [4]:
# get the information of km, vmax and public metabolites
km = pd.read_csv('./km.csv')
vmax = pd.read_csv('./vmax.csv')
public_metabolism = pd.read_csv('./public_metabolism_test.csv')

'''
public_metabolism_name_list = []
public_metabolism_concentration_list = []
for i in public_metabolism['metabolism']:
    public_metabolism_name_list.append(i)
for j in public_metabolism['concentration']:
    public_metabolism_concentration_list.append(j)
public_metabolism = dict(zip(public_metabolism_name_list, public_metabolism_concentration_list))
print(public_metabolism)
'''

"\npublic_metabolism_name_list = []\npublic_metabolism_concentration_list = []\nfor i in public_metabolism['metabolism']:\n    public_metabolism_name_list.append(i)\nfor j in public_metabolism['concentration']:\n    public_metabolism_concentration_list.append(j)\npublic_metabolism = dict(zip(public_metabolism_name_list, public_metabolism_concentration_list))\nprint(public_metabolism)\n"

In [5]:
# definate the function to simulate metabolism and interaction
def time_space_state(model_list, biomass_list, growth_list, breed_list, parameter_list, public_metabolism, vmax, km, deta_t, deta_x, micro_distribute_threshold, length, D, o2_concentration):
    
    number_cell_side = length // deta_x
    number_cell_side = int(number_cell_side)
    public_metabolism_name_list = []
    public_metabolism_concentration_list = []
    for i in public_metabolism['metabolism']:
        public_metabolism_name_list.append(i)
    for j in public_metabolism['concentration']:
        public_metabolism_concentration_list.append(j)
    public_metabolism = dict(zip(public_metabolism_name_list, public_metabolism_concentration_list))
    
    number_model = len(model_list)
    
    k_m = {}
    v_max = {}
    for i in range(number_model):
        for j in range(len(km['reactions_strain'+str(i)])):
            k_m[(i, km['reactions_strain'+str(i)][j])] = km['km_strain'+str(i)][j]
            v_max[(i, vmax['reactions_strain'+str(i)][j])] = vmax['vmax_strain'+str(i)][j]
    v_max_o2 = v_max[(0,'EX_glc__D_e_reverse')]
    k_m_o2 = k_m[(0,'EX_glc__D_e_reverse')]        
    
    number_public_metabolism = len(public_metabolism)
    distribute_micro_list = {}
    distribute_public_metabolism_list = {}
    distribute_lb_list = {}
    public_metabolism_r_list = []
    #set the initial distribution of straints
    for i in range(number_model):
        distribute_micro = np.random.randint(20, size=number_cell_side)
        distribute_micro_list.update({i: distribute_micro+1})
    print(distribute_micro_list)
    distribute_micro_list_initial = copy.deepcopy(distribute_micro_list)
    #set the initial distribution of metabolites
    for i in public_metabolism_name_list:
        distribute_public_metabolism = multiply(np.mat(np.ones(number_cell_side)), public_metabolism[i])
        distribute_public_metabolism_list.update({i: distribute_public_metabolism})
    #calculate the upperbounds of uptake reactions for public metabolites
    public_reaction_ub_list = {}
    public_reaction_list = {}
    for i in range(number_model):
        public_reaction_ub = {}
        public_reaction = []
        for rea in model_list[i]['reaction_list']:
            if 'EX_' in rea:
                for j in public_metabolism_name_list:
                    try:
                        model_list[i]['coef_matrix'][(j,rea)]
                    except:
                        pass
                    else:
                        ub = np.mat(np.ones(number_cell_side))
                        if model_list[i]['coef_matrix'][(j,rea)] > 0:
                            for m in range(number_cell_side):
                                ub[0,m] = v_max[(i,rea)]*distribute_public_metabolism_list[j][0,m]/(distribute_public_metabolism_list[j][0,m]+k_m[(i,rea)])
                        else:
                            try:
                                model_list[i]['ub_list'][rea]
                            except:
                                ub = ub * 1000
                            else:
                                ub = ub * model_list[i]['ub_list'][rea]
                        public_reaction_ub.update({rea: ub})
                        public_reaction.append(rea)
                        break          
            public_reaction_ub_list[i] = public_reaction_ub
            public_reaction_list[i] = public_reaction

            
    ct = 0
    
    distribute_micro_list_t = {ct: distribute_micro_list}
    distribute_public_metabolism_list_t = {ct: distribute_public_metabolism_list}
    distribute_lb_list_t = {ct: distribute_lb_list}
    r = micro_distribute_threshold + 1
    r_threshold = [r]*5
    slip_r = np.mean(r_threshold[-5:])
    number = {}
    various = {}
    for i in range(number_model):
        number[i] = [np.mean(distribute_micro_list[i])]
        various[i] = [np.std(distribute_micro_list[i])/np.mean(distribute_micro_list[i])]
    
            
    # iterative simulation by slip_r
    biomass = {}
    while slip_r >= micro_distribute_threshold:
        ct += 1
        print(ct)
        distribute_micro_list_t[ct] = copy.deepcopy(distribute_micro_list_t[ct-1])
        micro_decrease = {}
        micro_increase = {}
        
        #simulate the cell wandering
        #micro_decrease: the decrease number of strains
        #micro_increase: the increase number of strains
        #micro_decrease_cell: the decrease number of strains in the current grid
        #micro_increase_fcell: the increase number of strains in the forward grid
        #micro_increase_bcell: the increase number of strains in the backward grid
        for m in range(number_cell_side):
            micro_decrease_cell = {}
            micro_increase_cell = {}
            for i in range(number_model):
                micro_decrease_cell[i] = 0
                micro_increase_cell[i] = 0
            micro_decrease[m] = micro_decrease_cell
            micro_increase[m] = micro_increase_cell
            
            
        if ct > 0:
            met = 'glc__D_e'
            threshold = 0.3
            for m in range(number_cell_side):
                #calculate the number of strains in the internal grids
                if 0<m<number_cell_side-1:
                    for i in range(number_model):
                        if distribute_micro_list_t[ct][i][m] > 0:
                            if distribute_public_metabolism_list[met][0,m-1] > distribute_public_metabolism_list[met][0,m]:
                                if distribute_public_metabolism_list[met][0,m+1] > distribute_public_metabolism_list[met][0,m]:
                                    micro_decrease_cell_ratio = min(threshold,(distribute_public_metabolism_list[met][0,m+1]/(distribute_micro_list_t[ct][i][m+1]+0.001)+distribute_public_metabolism_list[met][0,m-1]/(distribute_micro_list_t[ct][i][m-1]+0.001)-2*distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001))/(distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001)+0.001))
                                    micro_decrease_cell = micro_decrease_cell_ratio * distribute_micro_list_t[ct][i][m]
                                    micro_decrease_cell = micro_decrease_cell
                                    micro_increase_fcell_ratio = (distribute_public_metabolism_list[met][0,m-1]/(distribute_micro_list_t[ct][i][m-1]+0.001)-distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001))/(distribute_public_metabolism_list[met][0,m+1]/(distribute_micro_list_t[ct][i][m+1]+0.001)+distribute_public_metabolism_list[met][0,m-1]/(distribute_micro_list_t[ct][i][m-1]+0.001)-2*distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001))
                                    micro_increase_fcell = micro_increase_fcell_ratio * micro_decrease_cell
                                    micro_increase_fcell = micro_increase_fcell
                                    micro_increase_bcell = micro_decrease_cell-micro_increase_fcell
                                    micro_decrease[m][i] = micro_decrease[m][i] + micro_decrease_cell
                                    micro_increase[m-1][i] = micro_increase[m-1][i] + micro_increase_fcell
                                    micro_increase[m+1][i] = micro_increase[m+1][i] + micro_increase_bcell
                                else:
                                    micro_decrease_cell_ratio = min(threshold,(distribute_public_metabolism_list[met][0,m-1]/(distribute_micro_list_t[ct][i][m-1]+0.001)-distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001))/(distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001)+0.001))
                                    micro_decrease_cell = micro_decrease_cell_ratio * distribute_micro_list_t[ct][i][m]
                                    micro_decrease_cell = micro_decrease_cell
                                    micro_increase_fcell_ratio = min(threshold,(distribute_public_metabolism_list[met][0,m-1]/(distribute_micro_list_t[ct][i][m-1]+0.001)-distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001))/(distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001)+0.001))
                                    micro_increase_fcell = micro_increase_fcell_ratio * distribute_micro_list_t[ct][i][m]
                                    micro_increase_fcell = micro_increase_fcell
                                    micro_decrease[m][i] = micro_decrease[m][i] + micro_decrease_cell
                                    micro_increase[m-1][i] = micro_increase[m-1][i] + micro_increase_fcell
                            elif distribute_public_metabolism_list[met][0,m+1] > distribute_public_metabolism_list[met][0,m]:
                                micro_decrease_cell_ratio = min(threshold,(distribute_public_metabolism_list[met][0,m+1]/(distribute_micro_list_t[ct][i][m+1]+0.001)-distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001))/(distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001)+0.001))
                                micro_decrease_cell = micro_decrease_cell_ratio * distribute_micro_list_t[ct][i][m]
                                micro_decrease_cell = micro_decrease_cell
                                micro_increase_bcell_ratio = min(threshold,(distribute_public_metabolism_list[met][0,m+1]/(distribute_micro_list_t[ct][i][m+1]+0.001)-distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001))/(distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001)+0.001))
                                micro_increase_bcell = micro_increase_bcell_ratio * distribute_micro_list_t[ct][i][m]
                                micro_increase_bcell = micro_increase_bcell
                                micro_decrease[m][i] = micro_decrease[m][i] + micro_decrease_cell
                                micro_increase[m+1][i] = micro_increase[m+1][i] + micro_increase_bcell
                #calculate the number of strains in the first grid
                elif m == 0:
                    for i in range(number_model):
                        if distribute_micro_list_t[ct][i][m] > 0:
                            if distribute_public_metabolism_list[met][0,m+1] > distribute_public_metabolism_list[met][0,m]:
                                micro_decrease_cell_ratio = min(threshold,(distribute_public_metabolism_list[met][0,m+1]/(distribute_micro_list_t[ct][i][m+1]+0.001)-distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001))/(distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001)+0.001))
                                micro_decrease_cell = micro_decrease_cell_ratio * distribute_micro_list_t[ct][i][m]
                                micro_decrease_cell = micro_decrease_cell
                                micro_increase_bcell_ratio = min(threshold,(distribute_public_metabolism_list[met][0,m+1]/(distribute_micro_list_t[ct][i][m+1]+0.001)-distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001))/(distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001)+0.001))
                                micro_increase_bcell = micro_increase_bcell_ratio * distribute_micro_list_t[ct][i][m]
                                micro_increase_bcell = micro_increase_bcell
                                micro_decrease[m][i] = micro_decrease[m][i] + micro_decrease_cell
                                micro_increase[m+1][i] = micro_increase[m+1][i] + micro_increase_bcell
                #calculate the number of strains in the last grid
                elif m == number_cell_side-1:
                    for i in range(number_model):
                        if distribute_micro_list_t[ct][i][m] > 0:
                            if distribute_public_metabolism_list[met][0,m-1] > distribute_public_metabolism_list[met][0,m]:
                                micro_decrease_cell_ratio = min(threshold,(distribute_public_metabolism_list[met][0,m-1]/(distribute_micro_list_t[ct][i][m-1]+0.001)-distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001))/(distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001)+0.001))
                                micro_decrease_cell = micro_decrease_cell_ratio * distribute_micro_list_t[ct][i][m]
                                micro_decrease_cell = micro_decrease_cell
                                micro_increase_fcell_ratio = min(threshold,(distribute_public_metabolism_list[met][0,m-1]/(distribute_micro_list_t[ct][i][m-1]+0.001)-distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001))/(distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001)+0.001))
                                micro_increase_fcell = micro_increase_fcell_ratio * distribute_micro_list_t[ct][i][m]
                                micro_increase_fcell = micro_increase_fcell
                                micro_decrease[m][i] = micro_decrease[m][i] + micro_decrease_cell
                                micro_increase[m-1][i] = micro_increase[m-1][i] + micro_increase_fcell
        
            #calculate the number of strains after wandering
            for m in range(number_cell_side):
                for i in range(number_model):
                    distribute_micro_list_t[ct][i][m] = distribute_micro_list_t[ct][i][m] - micro_decrease[m][i]
                    distribute_micro_list_t[ct][i][m] = distribute_micro_list_t[ct][i][m] + micro_increase[m][i]
                    distribute_micro_list_t[ct][i][m] = max(0, distribute_micro_list_t[ct][i][m])
                            
                            
        
        #simulate the substrate diffusion by Fick's second law
        for m in range(number_cell_side): 
            if 0<m<number_cell_side-1:
                for met in public_metabolism_name_list:
                    distribute_public_metabolism_list[met][0,m] = distribute_public_metabolism_list[met][0,m] + ((distribute_public_metabolism_list[met][0,m+1]-distribute_public_metabolism_list[met][0,m])/deta_x-(distribute_public_metabolism_list[met][0,m]-distribute_public_metabolism_list[met][0,m-1])/deta_x)/deta_x*D*deta_t
                    if distribute_public_metabolism_list[met][0,m] < 0:
                        print('Warning: the D is too high!')
                    distribute_public_metabolism_list[met][0,m] = max(0,distribute_public_metabolism_list[met][0,m])
            if m == 0:
                for met in public_metabolism_name_list:
                    distribute_public_metabolism_list[met][0,m] = distribute_public_metabolism_list[met][0,m] + (distribute_public_metabolism_list[met][0,m+1]-distribute_public_metabolism_list[met][0,m])/deta_x/deta_x*D*deta_t
                    distribute_public_metabolism_list[met][0,m] = max(0,distribute_public_metabolism_list[met][0,m])
            if m == number_cell_side-1:
                for met in public_metabolism_name_list:
                    distribute_public_metabolism_list[met][0,m] = distribute_public_metabolism_list[met][0,m] + (distribute_public_metabolism_list[met][0,m-1]-distribute_public_metabolism_list[met][0,m])/deta_x/deta_x*D*deta_t
                    distribute_public_metabolism_list[met][0,m] = max(0,distribute_public_metabolism_list[met][0,m])
        
        #simulate the metabolism by ETMs
        biomass_value_list = {}
        number_model_range = []
        biomass_strain = {}
        for m in range(number_cell_side):
            biomass_strain[m] = [0, 0]
            if m%2 == 0:
                number_model_range.append([0,1])
            elif m%2 == 1:
                number_model_range.append([1,0])
            
            
            biomass_value_micro = {}
            for i in number_model_range[m]:
                if distribute_micro_list_t[ct][i][m] > 0:
                    public_metabolism_flux_list = {}
                    for j in public_metabolism_name_list:
                        public_metabolism_flux_list.update({j: 0})
                    for j in public_reaction_list[i]:
                        model_list[i]['ub_list'][j] = public_reaction_ub_list[i][j][0,m]
                    model_list[i]['ub_list']['EX_o2_e_reverse'] = min(v_max_o2*o2_concentration/(o2_concentration+k_m_o2), model_list[i]['ub_list']['EX_o2_e_reverse'])   
            
                    biomass_id = biomass_list[i]
                    E_total=parameter_list[i][0]
                    #set the carbon source to glucose
                    substrate_name='EX_glc__D_e_reverse'
                    substrate_value=parameter_list[i][1]
                    biomass_value=growth_list[i]
                    K_value=parameter_list[i][2]

                    #calculate the MDF values of metabolic networks
                    biomass_value_micro[i] = biomass_value
                    obj_name=biomass_list[i]
                    obj_target='maximize'
                    if i == 0:
                        max_biomass_under_mdf=Max_Growth_Rate_Calculation0(model_list[i],obj_name,obj_target,substrate_name,substrate_value,K_value,E_total,0,'gurobi')
                    elif i == 1:
                        max_biomass_under_mdf=Max_Growth_Rate_Calculation1(model_list[i],obj_name,obj_target,substrate_name,substrate_value,K_value,E_total,0,'gurobi')
                    biomass_value=max_biomass_under_mdf*0.9
                        
                    #calculate the minimum value of the sum of the fluxes
                    if i == 0:
                        [min_V,Concretemodel]=Min_Flux_Sum_Calculation0(model_list[i],biomass_value,biomass_id,substrate_name,substrate_value,K_value,E_total,0,'gurobi')
                    elif i == 1:
                        [min_V,Concretemodel]=Min_Flux_Sum_Calculation1(model_list[i],biomass_value,biomass_id,substrate_name,substrate_value,K_value,E_total,0,'gurobi')
            
                    #translating the results of ETMs into a usable form
                    model=model_list[i]['model']
                    reaction_kcat_MW=model_list[i]['reaction_kcat_MW']
                    reaction_g0=model_list[i]['reaction_g0']
                    coef_matrix=model_list[i]['coef_matrix']
                    metabolite_list=model_list[i]['metabolite_list']
                    use_result = Get_Results_Thermodynamics(model,Concretemodel,reaction_kcat_MW,reaction_g0,coef_matrix,metabolite_list)
            
                    if distribute_micro_list_t[ct][i][m] == 0:
                        biomass_strain[m][i] = 0
                    else:
                        biomass_strain[m][i] = use_result['flux'][biomass_list[i]]
                
                    #simulate the fluxes of the public metabolites
                    for rea in public_reaction_list[i]:
                        for met in public_metabolism_name_list:
                            try:
                                model_list[i]['coef_matrix'][(met,rea)]  
                            except:
                                pass
                            else:
                                public_metabolism_flux_list[met] = public_metabolism_flux_list[met] + model_list[i]['coef_matrix'][(met,rea)]* use_result['flux'][rea]
                        
                    #simulate the distribution of the public metabolites
                    distribute_public_metabolism_ori = copy.deepcopy(distribute_public_metabolism_list)
                    distribute_public_metabolism_re = copy.deepcopy(distribute_public_metabolism_ori)
                    for met in public_metabolism_name_list:
                        distribute_public_metabolism_list[met][0,m] = distribute_public_metabolism_list[met][0,m] - public_metabolism_flux_list[met] * deta_t * distribute_micro_list_t[ct][i][m]
                        distribute_public_metabolism_ori[met][0,m] = copy.deepcopy(distribute_public_metabolism_list[met][0,m])
                        if distribute_public_metabolism_list[met][0,m] < 0:
                            print(met+'_1: ', distribute_public_metabolism_list[met][0,m])
                                
                    #simulate the multiplication and death rates of strains and the distribution of public metabolites after multiplication or death        
                    death_rate = 0
                    birth_rate = 1
                    for rea in public_reaction_list[i]:
                        if 'reverse' not in rea:
                            met = rea[3::]
                            if met in public_metabolism_name_list:
                            
                                intracellular_c = use_result['met_concentration'][rea].lstrip(';')
                                intracellular_c = intracellular_c.replace(';',',"')
                                intracellular_c = intracellular_c.replace(' :','" :')
                                intracellular_c = '{"' + intracellular_c + '}'
                                intracellular_c_dict = ast.literal_eval(intracellular_c)
                                
                                bd_rate_met = (distribute_public_metabolism_list[met][0,m] - 0.1)/intracellular_c_dict[met]/distribute_micro_list_t[ct][i][m]
                                if bd_rate_met < 0:
                                    death_rate_lb = (public_metabolism_flux_list[met] * deta_t * distribute_micro_list_t[ct][i][m] - distribute_public_metabolism_re[met][0,m] + 0.1)/(public_metabolism_flux_list[met] * deta_t * distribute_micro_list_t[ct][i][m] + intracellular_c_dict[met] * distribute_micro_list_t[ct][i][m])
                                    death_rate = max(death_rate, death_rate_lb)
                                else:
                                    birth_rate = min(birth_rate, bd_rate_met)
                    death_rate = min(death_rate, 1)
                    birth_rate = min(birth_rate,1)
                    if death_rate > 0:
                        birth_rate = 0
                        for met in public_metabolism_name_list:
                            distribute_public_metabolism_list[met][0,m] = distribute_public_metabolism_list[met][0,m] + public_metabolism_flux_list[met] * deta_t * math.ceil(distribute_micro_list_t[ct][i][m] * death_rate)
                            distribute_public_metabolism_ori[met][0,m] = copy.deepcopy(distribute_public_metabolism_list[met][0,m])
                    
                    for rea in public_reaction_list[i]:
                        if 'reverse' not in rea:
                            met = rea[3::]
                            if met in public_metabolism_name_list:
                            
                                intracellular_c = use_result['met_concentration'][rea].lstrip(';')
                                intracellular_c = intracellular_c.replace(';',',"')
                                intracellular_c = intracellular_c.replace(' :','" :')
                                intracellular_c = '{"' + intracellular_c + '}'
                                intracellular_c_dict = ast.literal_eval(intracellular_c)
                                
                                distribute_public_metabolism_list[met][0,m] = distribute_public_metabolism_ori[met][0,m] - intracellular_c_dict[met]*math.floor(distribute_micro_list_t[ct][i][m]*birth_rate/breed_list[i]) + intracellular_c_dict[met]*math.ceil(death_rate*distribute_micro_list_t[ct][i][m])
                                if distribute_public_metabolism_list[met][0,m] < 0:
                                    print(met+'_2: ', distribute_public_metabolism_list[met][0,m])
                                    distribute_public_metabolism_list[met][0,m] = 0
                        
                    #simulate the distribution of strains after multiplication or death
                    distribute_micro_increase = math.floor(distribute_micro_list_t[ct][i][m]*birth_rate/breed_list[i])
                    distribute_micro_decrease = math.ceil(distribute_micro_list_t[ct][i][m]*death_rate)
                    distribute_micro_list_t[ct][i][m] = max(0,distribute_micro_list_t[ct][i][m] + distribute_micro_increase - distribute_micro_decrease)
            
            #calculate the upperbounds of nutrient uptake rates after substrate diffusion, cell wandering, metabolism, multiplication and death
            biomass_value_list[m] = biomass_value_micro
            
            for i in range(number_model):
                for rea in model_list[i]['reaction_list']:
                    if 'EX_' in rea:
                        for j in public_metabolism_name_list:
                            try:
                                model_list[i]['coef_matrix'][(j,rea)]
                            except:
                                pass
                            else:
                                if model_list[i]['coef_matrix'][(j,rea)] > 0:
                                    public_reaction_ub_list[i][rea][0, m] = v_max[(i,rea)]*distribute_public_metabolism_list[j][0,m]/(distribute_public_metabolism_list[j][0,m]+k_m[(i,rea)])
                                    if i == 0:
                                        # gene upregulation for Fe3+ intake
                                        if 'EX_fe3_e' in rea:
                                            public_reaction_ub_list[i][rea][0, m] = public_reaction_ub_list[i][rea][0, m] * 1.2
                   
        
        if ct == 1:
            biomass_strain_initial = copy.deepcopy(biomass_strain)
        
        # calculate the mean number and uniformity of distribution of strains at this iteratio
        for i in range(number_model):
            strain_number = np.mean(distribute_micro_list_t[ct][i])
            strain_various = np.std(distribute_micro_list_t[ct][i])
            number[i].append(strain_number)
            various[i].append(strain_various)
            print('strain_number: ', i, strain_number)
            print('strain_various: ', i, strain_various)
        
        # calculate the slip_r at this iteration
        if ct > 1:
            r = 0
            for i in range(number_model):
                for m in range(number_cell_side):
                    if distribute_micro_list_t[ct-1][i][m] > 0:
                        r = r + abs((distribute_micro_list_t[ct][i][m]-distribute_micro_list_t[ct-1][i][m])/(distribute_micro_list_t[ct-1][i][m])/deta_t)
                    else:
                        r = r + abs(distribute_micro_list_t[ct][i][m]/deta_t)
       
        r_threshold.append(r)
        slip_r = np.mean(r_threshold[-1:])
        fd_r_threshold = r_threshold[1:]
        print('fd_r_threshold: ', fd_r_threshold)
        print('slip_r: ', slip_r)
        
        biomass_x = {}
        biomass_x_initial = {}
        for i in range(number_model):
            biomass_x[i] = [1]*number_cell_side
            if ct == 1:
                biomass_x_initial[i] = np.ones(number_cell_side)
            for m in range(number_cell_side):
                biomass_x[i][m] = distribute_micro_list_t[ct][i][m]*biomass_strain[m][i]
                if ct == 1:
                    biomass_x_initial[i][m] = distribute_micro_list_initial[i][m]*biomass_strain_initial[m][i]
        biomass[ct] = biomass_x
        
        if ct == 1:
            print('_1: ', distribute_micro_list_initial)
            print('_2: ', biomass_strain_initial)
            print('_3: ', biomass_x_initial)
            
    return(biomass_x_initial, biomass, number, various)

In [6]:
biomass_x_initial, biomass, number, various = time_space_state([model0, model1], ['BIOMASS_Ec_iML1515_core_75p37M', 'BIOMASS_SC5_notrace'], [0.7074, 0.28], [5, 30], [[0.13, 10, 1268], [200, 50, 3410]], public_metabolism, vmax, km, 1/15, 1, 0.1, 30, 0.1, 2.65*5)
print('The initial distribution of microorganism is: ',biomass_x_initial)
print('The change process of microorganism distribution is: ',biomass)
print('The mean numbers of microorganisms are: ', number)
print('The standard deviations of microorganisms are: ', various)

{0: array([ 3, 12,  3,  1, 18, 11,  7, 10, 18, 13, 11,  3, 20, 15, 14, 12, 18,
        7,  5,  7, 17,  3, 12,  8,  7, 14,  6, 11, 10, 19]), 1: array([18,  3,  5,  6, 20,  8,  4,  2, 12, 17,  9,  9, 14, 20,  2,  6, 15,
        9,  6, 18, 14,  2,  6, 16,  1,  9,  9, 13,  1,  1])}
1
strain_number:  0 12.2
strain_various:  0 6.379132647416367
strain_number:  1 9.166666666666666
strain_various:  1 6.000462945103116
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1]
slip_r:  1.1
_1:  {0: array([ 3, 12,  3,  1, 18, 11,  7, 10, 18, 13, 11,  3, 20, 15, 14, 12, 18,
        7,  5,  7, 17,  3, 12,  8,  7, 14,  6, 11, 10, 19]), 1: array([18,  3,  5,  6, 20,  8,  4,  2, 12, 17,  9,  9, 14, 20,  2,  6, 15,
        9,  6, 18, 14,  2,  6, 16,  1,  9,  9, 13,  1,  1])}
_2:  {0: [0.6784862515713712, 0.2857391744693639], 1: [0.6784862515713712, 0.2857391744693639], 2: [0.6784862515713712, 0.2857391744693639], 3: [0.6784862515713712, 0.2857391744693639], 4: [0.6784862515713712, 0.2857391744693639], 5: [0.678486251

glc__D_e_1:  -0.045849799815314896
glc__D_e_1:  -22.16408131131083
glc__D_e_1:  -21.08291116926295
glc__D_e_1:  -17.91756938072497
glc__D_e_1:  -7.167479935453879
glc__D_e_1:  -27.320819934219287
glc__D_e_1:  -6.344211192002225
glc__D_e_1:  -14.356801196567709
glc__D_e_1:  -9.294408084366024
glc__D_e_1:  -25.975543386474598
glc__D_e_1:  -10.742484357067323
glc__D_e_1:  -17.64270828757928
glc__D_e_1:  -17.079277840890114
glc__D_e_1:  -16.44501632000215
glc__D_e_1:  -25.999194740969806
glc__D_e_1:  -27.438612593006525
glc__D_e_1:  -23.396430855727665
glc__D_e_1:  -25.029772834145074
glc__D_e_1:  -21.046438201071307
glc__D_e_1:  -28.574424029041456
glc__D_e_1:  -25.64162261417461
glc__D_e_1:  -19.197223358711625
glc__D_e_1:  -28.91350792580901
glc__D_e_1:  -18.179529866340612
glc__D_e_1:  -17.78522034334034
strain_number:  0 36.63333333333333
strain_various:  0 18.017553169679346
strain_number:  1 0.1
strain_various:  1 0.3
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1, 855.275538187303, 340.

glc__D_e_1:  -4.820620259720978
glc__D_e_1:  -13.432692615868822
glc__D_e_1:  -7.977029622077024
glc__D_e_1:  -4.570632591712741
glc__D_e_1:  -2.3792443722767977
glc__D_e_1:  -1.3800962064066007
glc__D_e_1:  -0.9170709666999856
glc__D_e_1:  -0.21008332434820853
glc__D_e_1:  -0.2371475077805021
glc__D_e_1:  -0.2157585779549288
glc__D_e_1:  -0.4951950237803966
glc__D_e_1:  -0.11820680780613457
glc__D_e_1:  -0.5076021583649533
glc__D_e_1:  -1.6468307404006723
glc__D_e_1:  -1.7131221774298702
glc__D_e_1:  -1.3663885645306078
glc__D_e_1:  -1.8818052624341544
glc__D_e_1:  -0.4689678062121807
glc__D_e_1:  -0.9269095298909827
glc__D_e_1:  -0.683507297628972
glc__D_e_1:  -0.18960331525327634
glc__D_e_1:  -0.7222068661761634
strain_number:  0 2.7666666666666666
strain_various:  0 3.765486540792424
strain_number:  1 0.0
strain_various:  1 0.0
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1, 855.275538187303, 340.79755441926494, 388.4353442610023, 503.74473443930157, 450.71434069608944, 437.575585468813

In [7]:
biomass_x_initial, biomass, number, various = time_space_state([model0, model1], ['BIOMASS_Ec_iML1515_core_75p37M', 'BIOMASS_SC5_notrace'], [0.7074, 0.28], [5, 30], [[0.13, 10, 1268], [200, 50, 3410]], public_metabolism, vmax, km, 1/15, 1, 0.1, 30, 0.06, 2.65*3)
print('The initial distribution of microorganism is: ',biomass_x_initial)
print('The change process of microorganism distribution is: ',biomass)
print('The mean numbers of microorganisms are: ', number)
print('The standard deviations of microorganisms are: ', various)

{0: array([ 2, 16, 16, 19,  8,  8,  1, 11, 18,  9, 11, 11, 10,  4, 11, 10,  6,
       14,  8, 18,  7,  6, 17,  4, 18,  7, 19,  9,  8,  5]), 1: array([ 8, 16, 14, 12, 16, 19, 11, 11,  1, 17, 13, 20,  5, 14,  6,  7, 15,
        7,  8, 14,  3,  1,  2, 15, 18,  8, 20,  9,  9, 12])}
1
strain_number:  0 12.0
strain_various:  0 6.153589738247641
strain_number:  1 11.033333333333333
strain_various:  1 5.474080338792586
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1]
slip_r:  1.1
_1:  {0: array([ 2, 16, 16, 19,  8,  8,  1, 11, 18,  9, 11, 11, 10,  4, 11, 10,  6,
       14,  8, 18,  7,  6, 17,  4, 18,  7, 19,  9,  8,  5]), 1: array([ 8, 16, 14, 12, 16, 19, 11, 11,  1, 17, 13, 20,  5, 14,  6,  7, 15,
        7,  8, 14,  3,  1,  2, 15, 18,  8, 20,  9,  9, 12])}
_2:  {0: [0.6459265309330249, 0.2857391744693639], 1: [0.6459265309330249, 0.2857391744693639], 2: [0.6459265309330249, 0.2857391744693639], 3: [0.6459265309330249, 0.2857391744693639], 4: [0.6459265309330249, 0.2857391744693639], 5: [0.64592653

glc__D_e_1:  -11.58213860015336
glc__D_e_1:  -0.6536100729220609
glc__D_e_1:  -1.272006263779005
glc__D_e_1:  -12.282383993212532
glc__D_e_1:  -19.621845540380754
glc__D_e_1:  -1.6958181715217648
glc__D_e_1:  -2.4083513778177097
glc__D_e_1:  -16.618136575323476
glc__D_e_1:  -21.32075322458448
glc__D_e_1:  -2.5301653261459602
glc__D_e_1:  -0.38850038823534416
glc__D_e_1:  -4.46682860351235
glc__D_e_1:  -17.815801751972693
glc__D_e_1:  -1.4230089611257437
glc__D_e_1:  -1.0017757095550008
glc__D_e_1:  -20.54390668718117
glc__D_e_1:  -19.9119927422895
glc__D_e_1:  -0.4527307954184696
glc__D_e_1:  -1.077695637900308
glc__D_e_1:  -18.486888303478317
glc__D_e_1:  -24.20599960274655
glc__D_e_1:  -1.0115944514189312
glc__D_e_1:  -14.098454521275682
glc__D_e_1:  -26.11610419157724
glc__D_e_1:  -16.89948669047117
glc__D_e_1:  -15.935914145605661
glc__D_e_1:  -0.40768188433824193
glc__D_e_1:  -17.07792335835688
glc__D_e_1:  -22.81519214683299
glc__D_e_1:  -21.675893307955356
glc__D_e_1:  -32.49230

glc__D_e_1:  -1.431085657969138
glc__D_e_1:  -0.2456357464199252
glc__D_e_1:  -1.2265425667023944
glc__D_e_1:  -0.9881333604046383
glc__D_e_1:  -1.370226133266169
glc__D_e_1:  -0.9573661407403412
glc__D_e_1:  -1.6958807419552466
glc__D_e_1:  -3.474769831366672
glc__D_e_1:  -2.1886047857926836
glc__D_e_1:  -1.4414408089292747
glc__D_e_1:  -3.4248413747627557
glc__D_e_1:  -1.6599775691456704
glc__D_e_1:  -4.7592308618196775
glc__D_e_1:  -2.631168183449545
glc__D_e_1:  -1.3956722286196526
glc__D_e_1:  -5.001676269340646
glc__D_e_1:  -1.4806883425131805
glc__D_e_1:  -4.0946802663980835
glc__D_e_1:  -1.4297313974533414
glc__D_e_1:  -2.5510105840563626
glc__D_e_1:  -1.9290751403609465
glc__D_e_1:  -2.146501173121747
glc__D_e_1:  -0.6828116976150638
glc__D_e_1:  -1.9352245277508493
glc__D_e_1:  -0.7455789297117983
glc__D_e_1:  -3.3964557361911005
glc__D_e_1:  -0.9391581403235327
glc__D_e_1:  -0.6084766539191444
glc__D_e_1:  -1.6929071308509922
strain_number:  0 3.1666666666666665
strain_vario

In [8]:
biomass_x_initial, biomass, number, various = time_space_state([model0, model1], ['BIOMASS_Ec_iML1515_core_75p37M', 'BIOMASS_SC5_notrace'], [0.7074, 0.28], [5, 30], [[0.13, 10, 1268], [200, 50, 3410]], public_metabolism, vmax, km, 1/15, 1, 0.1, 30, 0.02, 2.65)
print('The initial distribution of microorganism is: ',biomass_x_initial)
print('The change process of microorganism distribution is: ',biomass)
print('The mean numbers of microorganisms are: ', number)
print('The standard deviations of microorganisms are: ', various)

{0: array([13,  8, 11,  7, 17,  3, 12,  4, 10, 13,  6,  8, 19,  2,  5,  8,  4,
       20, 17,  8, 10,  5,  4, 16,  1,  2,  5,  2, 19, 17]), 1: array([ 6, 15,  1, 17, 13,  4,  7,  3, 13, 13, 14, 17,  6,  1, 18, 13, 20,
       16, 16, 10,  6, 15, 12,  4, 18, 17, 16, 12, 20,  9])}
1
strain_number:  0 10.633333333333333
strain_various:  0 6.906438220160149
strain_number:  1 11.733333333333333
strain_various:  1 5.6091195585601685
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1]
slip_r:  1.1
_1:  {0: array([13,  8, 11,  7, 17,  3, 12,  4, 10, 13,  6,  8, 19,  2,  5,  8,  4,
       20, 17,  8, 10,  5,  4, 16,  1,  2,  5,  2, 19, 17]), 1: array([ 6, 15,  1, 17, 13,  4,  7,  3, 13, 13, 14, 17,  6,  1, 18, 13, 20,
       16, 16, 10,  6, 15, 12,  4, 18, 17, 16, 12, 20,  9])}
_2:  {0: [0.520733775712074, 0.2857391744693639], 1: [0.520733775712074, 0.2857391744693639], 2: [0.520733775712074, 0.2857391744693639], 3: [0.520733775712074, 0.2857391744693639], 4: [0.520733775712074, 0.2857391744693639], 5: [

glc__D_e_1:  -2.9603933597778926
glc__D_e_1:  -12.990441314609072
glc__D_e_1:  -14.108049090582115
glc__D_e_1:  -5.700670388517986
strain_number:  0 44.86666666666667
strain_various:  0 16.740635856767476
strain_number:  1 2.033333333333333
strain_various:  1 1.6428295373802142
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1, 860.1951809800107, 303.9638213308879, 343.1063869051484, 471.16376876477494, 529.3211975932564, 498.8227851826536, 522.8461085061892, 615.7898938137555, 527.223755789439, 416.08160570926105, 430.9017008141968, 453.83087319339074, 434.8609082511011, 329.09248258257213]
slip_r:  329.09248258257213
16
glc__D_e_1:  -12.352212467169842
glc__D_e_1:  -18.607065943281736
glc__D_e_1:  -11.630473701941554
glc__D_e_1:  -0.4652383587980943
glc__D_e_1:  -16.879089658358783
glc__D_e_1:  -18.711357098466085
glc__D_e_1:  -0.11937392732781726
glc__D_e_1:  -22.26503783059837
glc__D_e_1:  -33.61581562340309
glc__D_e_1:  -0.0020421000472670148
glc__D_e_1:  -22.66877987638235
glc__D_e_1:  -

glc__D_e_1:  -0.35898464302717237
glc__D_e_1:  -0.1956976685189198
glc__D_e_1:  -1.9243335088408504
glc__D_e_1:  -2.02285493295657
glc__D_e_1:  -0.7522425019327883
glc__D_e_1:  -1.9440250726629316
glc__D_e_1:  -1.3617180373770883
glc__D_e_1:  -1.3524641615625699
glc__D_e_1:  -2.018255025187681
glc__D_e_1:  -1.4954260908619394
glc__D_e_1:  -4.034202573643751
glc__D_e_1:  -0.3842536351884366
glc__D_e_1:  -0.5241813909590124
glc__D_e_1:  -1.518418560945813
glc__D_e_1:  -0.9210270303997687
glc__D_e_1:  -1.7019170036497524
glc__D_e_1:  -1.2053408589584724
glc__D_e_1:  -1.5972895108914154
glc__D_e_1:  -2.6380170802787033
glc__D_e_1:  -3.516006673519537
glc__D_e_1:  -3.34079907168361
glc__D_e_1:  -3.0490049219483755
glc__D_e_1:  -4.100453718872229
glc__D_e_1:  -0.8716876011316457
glc__D_e_1:  -0.627722174759735
glc__D_e_1:  -0.288619627773008
strain_number:  0 2.8
strain_various:  0 1.6812693617224654
strain_number:  1 0.0
strain_various:  1 0.0
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1, 860.

In [9]:
#30
biomass_x_initial, biomass, number, various = time_space_state([model0, model1], ['BIOMASS_Ec_iML1515_core_75p37M', 'BIOMASS_SC5_notrace'], [0.7074, 0.28], [5, 30], [[0.13, 10, 1268], [200, 50, 3410]], public_metabolism, vmax, km, 1/15, 1, 0.1, 30, 0.02, 2.45)
print('The initial distribution of microorganism is: ',biomass_x_initial)
print('The change process of microorganism distribution is: ',biomass)
print('The mean numbers of microorganisms are: ', number)
print('The standard deviations of microorganisms are: ', various)

{0: array([18, 12, 15, 17, 12, 19, 10, 10, 20, 13,  6, 10, 17,  7,  5,  7, 13,
        2, 18, 17,  5, 19, 19, 14, 13, 12, 17,  9,  6, 19]), 1: array([16,  8,  2, 18,  5, 15, 18,  5, 14, 10,  6,  4,  4, 13, 20, 14, 12,
       15,  7,  1, 19, 20,  9, 17,  2, 19, 16,  1, 17, 11])}
1
strain_number:  0 14.833333333333334
strain_various:  0 5.972622725589004
strain_number:  1 11.266666666666667
strain_various:  1 6.191032941996746
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1]
slip_r:  1.1
_1:  {0: array([18, 12, 15, 17, 12, 19, 10, 10, 20, 13,  6, 10, 17,  7,  5,  7, 13,
        2, 18, 17,  5, 19, 19, 14, 13, 12, 17,  9,  6, 19]), 1: array([16,  8,  2, 18,  5, 15, 18,  5, 14, 10,  6,  4,  4, 13, 20, 14, 12,
       15,  7,  1, 19, 20,  9, 17,  2, 19, 16,  1, 17, 11])}
_2:  {0: [0.5090172841113372, 0.2857391744693639], 1: [0.5090172841113372, 0.2857391744693639], 2: [0.5090172841113372, 0.2857391744693639], 3: [0.5090172841113372, 0.2857391744693639], 4: [0.5090172841113372, 0.2857391744693639], 

glc__D_e_1:  -2.84568444478672
glc__D_e_1:  -8.195582218014668
glc__D_e_1:  -25.52376932447011
glc__D_e_1:  -3.062780646560004
glc__D_e_1:  -0.02318775294503861
glc__D_e_1:  -12.173335852652292
glc__D_e_1:  -36.722653663923154
glc__D_e_1:  -1.449763210487525
glc__D_e_1:  -0.04931882765497986
glc__D_e_1:  -9.609949597929099
glc__D_e_1:  -23.86358625955742
glc__D_e_1:  -1.5831529506689197
glc__D_e_1:  -1.1330519801879202
glc__D_e_1:  -17.944331272874813
glc__D_e_1:  -31.374275226479032
glc__D_e_1:  -2.1729986365899094
glc__D_e_1:  -2.338471377645123
glc__D_e_1:  -20.703461067991896
glc__D_e_1:  -32.32752099446869
glc__D_e_1:  -1.6083409386929404
glc__D_e_1:  -1.029542552250755
glc__D_e_1:  -21.466146106144038
glc__D_e_1:  -20.580476486602787
glc__D_e_1:  -2.673198206352563
glc__D_e_1:  -2.556652784813416
glc__D_e_1:  -26.618580599913088
strain_number:  0 40.8
strain_various:  0 19.109160107131867
strain_number:  1 2.3333333333333335
strain_various:  1 1.4452988925785866
fd_r_threshold:  

glc__D_e_1:  -2.42632024328346
glc__D_e_1:  -3.348661541251893
glc__D_e_1:  -1.7420781655441937
glc__D_e_1:  -1.9728354301993183
glc__D_e_1:  -1.433817767651828
glc__D_e_1:  -1.3476277742308536
glc__D_e_1:  -2.3891957547805602
glc__D_e_1:  -3.609763366306952
glc__D_e_1:  -1.913447645645323
glc__D_e_1:  -7.372931327304762
glc__D_e_1:  -6.44391395553151
glc__D_e_1:  -7.752478848202174
glc__D_e_1:  -4.906844284948732
glc__D_e_1:  -7.03701081621928
glc__D_e_1:  -4.448965140254884
glc__D_e_1:  -2.468140330197056
glc__D_e_1:  -3.774474993515411
glc__D_e_1:  -2.292922056292915
glc__D_e_1:  -2.828029945361174
glc__D_e_1:  -1.446312274678249
glc__D_e_1:  -2.2475091842122117
glc__D_e_1:  -2.9103666499245424
glc__D_e_1:  -2.7162411506722206
glc__D_e_1:  -1.393732710392832
glc__D_e_1:  -2.995942854819052
glc__D_e_1:  -2.400184751422262
glc__D_e_1:  -2.6645133852699083
glc__D_e_1:  -2.3336225230751424
glc__D_e_1:  -0.28450657418576897
strain_number:  0 4.766666666666667
strain_various:  0 2.6793448

In [10]:
#30
biomass_x_initial, biomass, number, various = time_space_state([model0, model1], ['BIOMASS_Ec_iML1515_core_75p37M', 'BIOMASS_SC5_notrace'], [0.7074, 0.28], [5, 30], [[0.13, 10, 1268], [200, 50, 3410]], public_metabolism, vmax, km, 1/15, 1, 0.1, 30, 0.06, 2.45*3)
print('The initial distribution of microorganism is: ',biomass_x_initial)
print('The change process of microorganism distribution is: ',biomass)
print('The mean numbers of microorganisms are: ', number)
print('The standard deviations of microorganisms are: ', various)

{0: array([ 7, 11, 15,  6,  1, 12,  7,  8, 15,  6,  8, 19, 20,  9,  6,  8, 19,
       20, 19,  5, 14, 20, 14,  1,  6, 14,  2, 10, 11,  5]), 1: array([ 1,  2, 11,  1, 10, 13,  2, 20,  2, 17, 13,  6, 20,  3, 18,  2,  4,
       16,  3, 10, 19, 17, 16,  2, 11, 17,  2, 14, 11, 11])}
1
strain_number:  0 12.366666666666667
strain_various:  0 6.906438220160149
strain_number:  1 9.8
strain_various:  1 6.574698979167539
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1]
slip_r:  1.1
_1:  {0: array([ 7, 11, 15,  6,  1, 12,  7,  8, 15,  6,  8, 19, 20,  9,  6,  8, 19,
       20, 19,  5, 14, 20, 14,  1,  6, 14,  2, 10, 11,  5]), 1: array([ 1,  2, 11,  1, 10, 13,  2, 20,  2, 17, 13,  6, 20,  3, 18,  2,  4,
       16,  3, 10, 19, 17, 16,  2, 11, 17,  2, 14, 11, 11])}
_2:  {0: [0.5090172841113372, 0.2857391744693639], 1: [0.5090172841113372, 0.2857391744693639], 2: [0.5090172841113372, 0.2857391744693639], 3: [0.5090172841113372, 0.2857391744693639], 4: [0.5090172841113372, 0.2857391744693639], 5: [0.509017284

glc__D_e_1:  -0.41686824234630215
glc__D_e_1:  -0.21953845485251433
glc__D_e_1:  -13.923396952219647
glc__D_e_1:  -17.122221483109968
glc__D_e_1:  -15.475040435785314
glc__D_e_1:  -16.28677368840247
glc__D_e_1:  -0.2585096120820829
glc__D_e_1:  -7.872228196351568
strain_number:  0 44.1
strain_various:  0 18.92150452087078
strain_number:  1 1.5
strain_various:  1 1.4776106839534333
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1, 849.738814977592, 270.30995707079296, 301.5004819741662, 351.8166809916174, 381.5379399683343, 455.683843568712, 442.9641489805376, 420.5598710878318, 378.65763084819645, 342.5598762064255, 311.9692903406433, 363.10212164633367, 369.40729857830615]
slip_r:  369.40729857830615
15
glc__D_e_1:  -4.687900982292405
glc__D_e_1:  -17.448443735135314
glc__D_e_1:  -13.983045808103135
glc__D_e_1:  -16.101936112842104
glc__D_e_1:  -22.71117653905879
glc__D_e_1:  -6.6669484392399765
glc__D_e_1:  -22.84009793478559
glc__D_e_1:  -5.0726211690012475
glc__D_e_1:  -18.731710958158345

glc__D_e_1:  -8.766333350831019
glc__D_e_1:  -9.393057405686028
glc__D_e_1:  -4.901434208823426
glc__D_e_1:  -8.350632246667233
glc__D_e_1:  -7.184432300388729
glc__D_e_1:  -5.029131468450819
glc__D_e_1:  -3.2247366984760046
glc__D_e_1:  -1.0908593306788226
glc__D_e_1:  -0.6966164465248575
glc__D_e_1:  -0.12532633677544158
glc__D_e_1:  -1.6663225064787808
glc__D_e_1:  -1.2003470425329157
glc__D_e_1:  -0.4185710567356089
glc__D_e_1:  -0.8434896493113264
glc__D_e_1:  -0.5081072340556092
glc__D_e_1:  -0.6375490851312221
glc__D_e_1:  -0.045657694873811994
glc__D_e_1:  -0.5782818903965052
glc__D_e_1:  -0.6115024570013363
glc__D_e_1:  -1.8040357408392413
glc__D_e_1:  -0.5970303511292462
glc__D_e_1:  -2.7903405843316085
glc__D_e_1:  -2.852087051635201
glc__D_e_1:  -3.8880746987189356
glc__D_e_1:  -5.443036215909366
glc__D_e_1:  -9.877504915357623
glc__D_e_1:  -5.72659205704737
strain_number:  0 4.766666666666667
strain_various:  0 3.9890126876486898
strain_number:  1 0.0
strain_various:  1 0.

In [6]:
#30
biomass_x_initial, biomass, number, various = time_space_state([model0, model1], ['BIOMASS_Ec_iML1515_core_75p37M', 'BIOMASS_SC5_notrace'], [0.7074, 0.28], [5, 30], [[0.13, 10, 1268], [200, 50, 3410]], public_metabolism, vmax, km, 1/15, 1, 0.1, 30, 0.1, 2.45*5)
print('The initial distribution of microorganism is: ',biomass_x_initial)
print('The change process of microorganism distribution is: ',biomass)
print('The mean numbers of microorganisms are: ', number)
print('The standard deviations of microorganisms are: ', various)

{0: array([ 4, 19,  4, 18, 11, 18,  9,  9, 12,  7, 18,  7, 13,  2, 19,  6,  5,
        9,  3, 15, 13, 12,  7, 16, 18, 12,  3, 14,  2, 19]), 1: array([16, 20,  9, 11, 20,  3, 18,  8, 15, 13,  7, 10, 13, 20, 12,  4,  5,
       12,  5,  5,  8,  4,  2,  8, 12,  7,  1, 10,  4,  3])}
1
strain_number:  0 12.433333333333334
strain_various:  0 6.736385446876455
strain_number:  1 9.5
strain_various:  1 5.493936051077891
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1]
slip_r:  1.1
_1:  {0: array([ 4, 19,  4, 18, 11, 18,  9,  9, 12,  7, 18,  7, 13,  2, 19,  6,  5,
        9,  3, 15, 13, 12,  7, 16, 18, 12,  3, 14,  2, 19]), 1: array([16, 20,  9, 11, 20,  3, 18,  8, 15, 13,  7, 10, 13, 20, 12,  4,  5,
       12,  5,  5,  8,  4,  2,  8, 12,  7,  1, 10,  4,  3])}
_2:  {0: [0.6741983849766484, 0.2857391744693639], 1: [0.6741983849766484, 0.2857391744693639], 2: [0.6741983849766484, 0.2857391744693639], 3: [0.6741983849766484, 0.2857391744693639], 4: [0.6741983849766484, 0.2857391744693639], 5: [0.674198384

glc__D_e_1:  -21.117546477105698
glc__D_e_1:  -17.45427980756998
glc__D_e_1:  -0.4628959703883879
glc__D_e_1:  -17.699530345442327
glc__D_e_1:  -1.782354282723471
glc__D_e_1:  -0.7335877723465676
glc__D_e_1:  -5.28355606723628
glc__D_e_1:  -6.75072601975203
glc__D_e_1:  -5.301856435313283
glc__D_e_1:  -12.135585924685671
glc__D_e_1:  -14.212695489155351
glc__D_e_1:  -2.8923972437104624
glc__D_e_1:  -0.4402470030456236
glc__D_e_1:  -13.1406530288812
strain_number:  0 47.3
strain_various:  0 20.49089228576117
strain_number:  1 1.4666666666666666
strain_various:  1 1.5860503004493758
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1, 691.3770374362479, 296.4828835380306, 325.9352938727939, 384.5759736780248, 441.86012729530523, 403.7971827793809, 351.37828877993354, 427.6756200118155, 388.7175724723233, 305.62326528971835, 331.6545443872733, 309.519726793191, 284.70009613842393]
slip_r:  284.70009613842393
15
glc__D_e_1:  -6.194592328019428
glc__D_e_1:  -0.5636428341308708
glc__D_e_1:  -0.7704144

glc__D_e_1:  -1.8045814512083371
glc__D_e_1:  -5.226742576092579
strain_number:  0 5.333333333333333
strain_various:  0 3.524517303436347
strain_number:  1 0.03333333333333333
strain_various:  1 0.17950549357115012
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1, 691.3770374362479, 296.4828835380306, 325.9352938727939, 384.5759736780248, 441.86012729530523, 403.7971827793809, 351.37828877993354, 427.6756200118155, 388.7175724723233, 305.62326528971835, 331.6545443872733, 309.519726793191, 284.70009613842393, 284.0686814832446, 264.2701834551233, 242.8934621854593, 239.56775085800624, 202.83802890657734]
slip_r:  202.83802890657734
20
glc__D_e_1:  -0.3319810938242792
glc__D_e_1:  -1.1185865852475467
glc__D_e_1:  -0.05230652662483998
glc__D_e_1:  -0.4630183684561142
glc__D_e_1:  -0.30278254240813973
glc__D_e_1:  -0.3166865447711955
glc__D_e_1:  -0.3814224446622406
glc__D_e_1:  -0.08585187593563282
glc__D_e_1:  -0.6067008098632619
glc__D_e_1:  -0.22310163294322827
glc__D_e_1:  -2.34134729155912

In [7]:
#35
biomass_x_initial, biomass, number, various = time_space_state([model0, model1], ['BIOMASS_Ec_iML1515_core_75p37M', 'BIOMASS_SC5_notrace'], [0.7074, 0.28], [5, 30], [[0.13, 10, 1268], [200, 50, 3410]], public_metabolism, vmax, km, 1/15, 1, 0.1, 30, 0.02, 2.29)
print('The initial distribution of microorganism is: ',biomass_x_initial)
print('The change process of microorganism distribution is: ',biomass)
print('The mean numbers of microorganisms are: ', number)
print('The standard deviations of microorganisms are: ', various)

{0: array([19,  6,  2,  6, 11,  4, 12,  6, 10, 19, 15,  1, 12,  5,  5, 17,  2,
       16, 15, 20, 10, 13, 14, 10,  9, 18, 19, 20, 13, 13]), 1: array([12, 10,  1, 13, 18, 11, 11,  6, 11,  7,  2,  5,  7, 18,  6,  8,  7,
       16, 14, 10, 15,  7, 16, 16, 20, 19, 13,  9, 11, 17])}
1
strain_number:  0 13.333333333333334
strain_various:  0 6.803593821568781
strain_number:  1 11.2
strain_various:  1 4.942334131426837
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1]
slip_r:  1.1
_1:  {0: array([19,  6,  2,  6, 11,  4, 12,  6, 10, 19, 15,  1, 12,  5,  5, 17,  2,
       16, 15, 20, 10, 13, 14, 10,  9, 18, 19, 20, 13, 13]), 1: array([12, 10,  1, 13, 18, 11, 11,  6, 11,  7,  2,  5,  7, 18,  6,  8,  7,
       16, 14, 10, 15,  7, 16, 16, 20, 19, 13,  9, 11, 17])}
_2:  {0: [0.49885752915679404, 0.2857391744693639], 1: [0.49885752915679404, 0.2857391744693639], 2: [0.49885752915679404, 0.2857391744693639], 3: [0.49885752915679404, 0.2857391744693639], 4: [0.49885752915679404, 0.2857391744693639], 5: [0.498

glc__D_e_1:  -2.44120517127733
glc__D_e_1:  -3.778687766658011
glc__D_e_1:  -16.257604361707806
strain_number:  0 42.3
strain_various:  0 18.756598838808706
strain_number:  1 2.433333333333333
strain_various:  1 1.2297244497131143
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1, 734.2057126310222, 314.8044050247997, 325.35438825880004, 420.8662403332046, 442.06993764312693, 362.2849807483721, 388.90035971742293, 356.94564151922594, 301.6699241565668, 323.782005085352, 323.70523984521265, 334.3577250565502]
slip_r:  334.3577250565502
14
glc__D_e_1:  -5.227411471047166
glc__D_e_1:  -18.640641809636854
glc__D_e_1:  -23.057284662976244
glc__D_e_1:  -23.910198098154684
glc__D_e_1:  -0.2533247524972886
glc__D_e_1:  -0.3194861924289361
glc__D_e_1:  -12.022584967325335
glc__D_e_1:  -23.645630190039565
glc__D_e_1:  -0.6574308928146806
glc__D_e_1:  -6.064453701181832
glc__D_e_1:  -23.96010831544483
glc__D_e_1:  -2.309257115084968
glc__D_e_1:  -7.670287580904023
glc__D_e_1:  -16.782295633149488
glc__D_

glc__D_e_1:  -2.357546757398538
glc__D_e_1:  -3.0569065350405955
glc__D_e_1:  -3.4063375812513943
glc__D_e_1:  -2.3075203388503174
glc__D_e_1:  -1.5430562765993812
glc__D_e_1:  -1.4158974094007286
glc__D_e_1:  -1.3082633672010253
glc__D_e_1:  -0.5457993407751272
glc__D_e_1:  -1.397481006112098
glc__D_e_1:  -0.2604980459628946
glc__D_e_1:  -1.2594147148435448
glc__D_e_1:  -0.09953605087089601
glc__D_e_1:  -0.12158422432166216
glc__D_e_1:  -0.9788433622235184
strain_number:  0 5.766666666666667
strain_various:  0 4.800578668822703
strain_number:  1 0.0
strain_various:  1 0.0
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1, 734.2057126310222, 314.8044050247997, 325.35438825880004, 420.8662403332046, 442.06993764312693, 362.2849807483721, 388.90035971742293, 356.94564151922594, 301.6699241565668, 323.782005085352, 323.70523984521265, 334.3577250565502, 373.68791528974657, 359.7046690831369, 353.55436797177447, 238.27491672502964, 238.50749557460085, 231.86616161616163]
slip_r:  231.8661616161616

In [8]:
#35
biomass_x_initial, biomass, number, various = time_space_state([model0, model1], ['BIOMASS_Ec_iML1515_core_75p37M', 'BIOMASS_SC5_notrace'], [0.7074, 0.28], [5, 30], [[0.13, 10, 1268], [200, 50, 3410]], public_metabolism, vmax, km, 1/15, 1, 0.1, 30, 0.06, 2.29*3)
print('The initial distribution of microorganism is: ',biomass_x_initial)
print('The change process of microorganism distribution is: ',biomass)
print('The mean numbers of microorganisms are: ', number)
print('The standard deviations of microorganisms are: ', various)

{0: array([12, 16,  6,  3,  6,  9,  1,  1, 14,  2, 20, 11, 18,  8, 18, 17, 19,
       11, 15, 19, 10,  7, 15, 17, 17, 20,  4, 20, 13, 17]), 1: array([ 8, 18,  8, 18,  4,  4,  3,  3, 16, 10, 13, 11,  1,  2,  4, 11, 18,
       12,  9,  3, 10, 13,  5,  5,  5,  6,  9, 15, 16, 19])}
1
strain_number:  0 14.266666666666667
strain_various:  0 7.3345453543867025
strain_number:  1 9.3
strain_various:  1 5.4169487106057534
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1]
slip_r:  1.1
_1:  {0: array([12, 16,  6,  3,  6,  9,  1,  1, 14,  2, 20, 11, 18,  8, 18, 17, 19,
       11, 15, 19, 10,  7, 15, 17, 17, 20,  4, 20, 13, 17]), 1: array([ 8, 18,  8, 18,  4,  4,  3,  3, 16, 10, 13, 11,  1,  2,  4, 11, 18,
       12,  9,  3, 10, 13,  5,  5,  5,  6,  9, 15, 16, 19])}
_2:  {0: [0.49885752915679404, 0.2857391744693639], 1: [0.49885752915679404, 0.2857391744693639], 2: [0.49885752915679404, 0.2857391744693639], 3: [0.49885752915679404, 0.2857391744693639], 4: [0.49885752915679404, 0.2857391744693639], 5: [0.49

glc__D_e_1:  -1.6449668423816775
glc__D_e_1:  -1.2083996645196684
glc__D_e_1:  -6.146991181718018
strain_number:  0 38.93333333333333
strain_various:  0 14.608064743680305
strain_number:  1 0.6
strain_various:  1 0.8406346808612327
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1, 599.7492102196051, 356.4022878437353, 439.2737906013835, 476.2359931734932, 413.18242101328906, 419.03066646751995, 468.2160975748225, 444.8545907112912, 435.2938040347179, 408.5563181735822, 386.60937768770594, 338.3986147848229]
slip_r:  338.3986147848229
14
glc__D_e_1:  -7.703125809209407
glc__D_e_1:  -29.626517198109514
glc__D_e_1:  -20.847143769578274
glc__D_e_1:  -14.265397712630337
glc__D_e_1:  -22.204001901934735
glc__D_e_1:  -18.300373900570232
glc__D_e_1:  -14.761811204632481
glc__D_e_1:  -20.836006257379104
glc__D_e_1:  -8.328584742522352
glc__D_e_1:  -13.87410604129745
glc__D_e_1:  -33.20335641252651
glc__D_e_1:  -14.058871892312977
glc__D_e_1:  -29.420549573435277
glc__D_e_1:  -8.585558695040621
glc__D_

glc__D_e_1:  -2.2892126611925194
glc__D_e_1:  -5.596968853204511
glc__D_e_1:  -6.292840796679851
glc__D_e_1:  -5.827435073284421
glc__D_e_1:  -15.69472532319215
glc__D_e_1:  -14.037445122237777
glc__D_e_1:  -14.832192484183874
glc__D_e_1:  -17.198525914686822
glc__D_e_1:  -3.7465497084787245
glc__D_e_1:  -3.0134447916209766
glc__D_e_1:  -1.5217657403855767
glc__D_e_1:  -0.9072959301340273
glc__D_e_1:  -1.0213067475568276
glc__D_e_1:  -0.1217055818281626
glc__D_e_1:  -0.39425384134546404
glc__D_e_1:  -0.39394579478641045
glc__D_e_1:  -0.5035134762938146
glc__D_e_1:  -0.09507957193015854
strain_number:  0 4.866666666666666
strain_various:  0 6.8153421696117995
strain_number:  1 0.0
strain_various:  1 0.0
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1, 599.7492102196051, 356.4022878437353, 439.2737906013835, 476.2359931734932, 413.18242101328906, 419.03066646751995, 468.2160975748225, 444.8545907112912, 435.2938040347179, 408.5563181735822, 386.60937768770594, 338.3986147848229, 285.7773593836

In [9]:
#35
biomass_x_initial, biomass, number, various = time_space_state([model0, model1], ['BIOMASS_Ec_iML1515_core_75p37M', 'BIOMASS_SC5_notrace'], [0.7074, 0.28], [5, 30], [[0.13, 10, 1268], [200, 50, 3410]], public_metabolism, vmax, km, 1/15, 1, 0.1, 30, 0.1, 2.29*5)
print('The initial distribution of microorganism is: ',biomass_x_initial)
print('The change process of microorganism distribution is: ',biomass)
print('The mean numbers of microorganisms are: ', number)
print('The standard deviations of microorganisms are: ', various)

{0: array([ 1,  6, 19,  1,  9,  7, 20, 20, 13,  4, 18, 12, 15,  8, 13, 11, 16,
        3,  7, 11, 13, 14, 14, 11, 16,  5,  5,  5, 19,  6]), 1: array([19, 16,  2, 13,  2,  5, 18,  2, 15, 10, 17, 10,  2, 18,  9, 11,  2,
       19, 12, 18,  1, 12, 19, 16,  6,  1, 19,  5,  2,  4])}
1
strain_number:  0 12.5
strain_various:  0 6.711929677819934
strain_number:  1 10.166666666666666
strain_various:  1 6.653737462676313
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1]
slip_r:  1.1
_1:  {0: array([ 1,  6, 19,  1,  9,  7, 20, 20, 13,  4, 18, 12, 15,  8, 13, 11, 16,
        3,  7, 11, 13, 14, 14, 11, 16,  5,  5,  5, 19,  6]), 1: array([19, 16,  2, 13,  2,  5, 18,  2, 15, 10, 17, 10,  2, 18,  9, 11,  2,
       19, 12, 18,  1, 12, 19, 16,  6,  1, 19,  5,  2,  4])}
_2:  {0: [0.49885752915679404, 0.2857391744693639], 1: [0.49885752915679404, 0.2857391744693639], 2: [0.49885752915679404, 0.2857391744693639], 3: [0.49885752915679404, 0.2857391744693639], 4: [0.49885752915679404, 0.2857391744693639], 5: [0.498

glc__D_e_1:  -22.870828263314824
glc__D_e_1:  -34.87001738533458
glc__D_e_1:  -1.7250665363622737
glc__D_e_1:  -1.3832564371066265
glc__D_e_1:  -17.7694756211786
glc__D_e_1:  -17.38083727778276
glc__D_e_1:  -1.1426988775128635
glc__D_e_1:  -29.069103189107743
glc__D_e_1:  -29.3964444156186
glc__D_e_1:  -0.9955323183037486
glc__D_e_1:  -25.480108611905862
glc__D_e_1:  -17.139400026959237
glc__D_e_1:  -14.133287691787253
strain_number:  0 41.96666666666667
strain_various:  0 18.409206633879933
strain_number:  1 1.6666666666666667
strain_various:  1 1.4220486005134358
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1, 923.8654520144457, 343.79518763400347, 346.45345553569234, 385.69682313300734, 390.32396228642364, 413.52630409451234, 428.49786943390507, 397.87742923833076, 382.3231061971258, 370.56023334231463, 320.40325895426105, 348.35722590130626, 395.1573632827917]
slip_r:  395.1573632827917
15
glc__D_e_1:  -3.73549415969309
glc__D_e_1:  -24.978608677459334
glc__D_e_1:  -17.24601031652903
gl

glc__D_e_1:  -13.204684931348313
glc__D_e_1:  -0.19023857060525096
glc__D_e_1:  -2.4792566572825496
glc__D_e_1:  -9.685533777255413
glc__D_e_1:  -2.465369839572981
glc__D_e_1:  -2.081769816266304
glc__D_e_1:  -1.4071945568731299
glc__D_e_1:  -0.3096075810187747
glc__D_e_1:  -0.040139175325752896
glc__D_e_1:  -0.3815728877625184
glc__D_e_1:  -1.6739495463684164
glc__D_e_1:  -0.16383617808138784
glc__D_e_1:  -0.5866093745161907
glc__D_e_1:  -0.6555611363059204
glc__D_e_1:  -0.799866543059078
glc__D_e_1:  -1.2777598342226848
glc__D_e_1:  -0.40938371107761795
glc__D_e_1:  -1.166499861730849
glc__D_e_1:  -2.2719118181958207
glc__D_e_1:  -3.553480306647532
glc__D_e_1:  -2.783484649117262
glc__D_e_1:  -3.0804669669912226
glc__D_e_1:  -6.815045145889647
glc__D_e_1:  -1.9493388950075614
strain_number:  0 3.533333333333333
strain_various:  0 3.955867653105812
strain_number:  1 0.0
strain_various:  1 0.0
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1, 923.8654520144457, 343.79518763400347, 346.4534555

In [10]:
#35
biomass_x_initial, biomass, number, various = time_space_state([model0, model1], ['BIOMASS_Ec_iML1515_core_75p37M', 'BIOMASS_SC5_notrace'], [0.7074, 0.28], [5, 30], [[0.13, 10, 1268], [200, 50, 3410]], public_metabolism, vmax, km, 1/15, 1, 0.1, 30, 0.06, 2.29)
print('The initial distribution of microorganism is: ',biomass_x_initial)
print('The change process of microorganism distribution is: ',biomass)
print('The mean numbers of microorganisms are: ', number)
print('The standard deviations of microorganisms are: ', various)

{0: array([ 8,  1, 18,  4,  7,  8, 17, 16, 14,  5,  6, 19, 15, 16,  3,  4, 17,
       14,  4, 11, 18,  2, 10,  2,  1,  7, 18,  5,  2, 13]), 1: array([ 4, 18,  2, 17, 14, 19, 15,  6,  4, 20, 17, 11, 14,  5, 12,  6,  4,
        1,  3,  6, 19,  3,  9, 16, 14, 16, 15,  1,  9, 17])}
1
strain_number:  0 10.966666666666667
strain_various:  0 7.250210724906935
strain_number:  1 10.566666666666666
strain_various:  1 6.178906771338186
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1]
slip_r:  1.1
_1:  {0: array([ 8,  1, 18,  4,  7,  8, 17, 16, 14,  5,  6, 19, 15, 16,  3,  4, 17,
       14,  4, 11, 18,  2, 10,  2,  1,  7, 18,  5,  2, 13]), 1: array([ 4, 18,  2, 17, 14, 19, 15,  6,  4, 20, 17, 11, 14,  5, 12,  6,  4,
        1,  3,  6, 19,  3,  9, 16, 14, 16, 15,  1,  9, 17])}
_2:  {0: [0.49885752915679404, 0.2857391744693639], 1: [0.49885752915679404, 0.2857391744693639], 2: [0.49885752915679404, 0.2857391744693639], 3: [0.49885752915679404, 0.2857391744693639], 4: [0.49885752915679404, 0.28573917446936

glc__D_e_1:  -8.42830594420581
glc__D_e_1:  -6.95786285354193
glc__D_e_1:  -20.80133246585057
glc__D_e_1:  -14.128166352204794
glc__D_e_1:  -22.99134872276987
glc__D_e_1:  -28.6028967242828
glc__D_e_1:  -19.70148348374783
glc__D_e_1:  -3.4712712543240656
glc__D_e_1:  -15.515982579104243
strain_number:  0 43.6
strain_various:  0 17.751244087856676
strain_number:  1 1.0666666666666667
strain_various:  1 1.2892719737209146
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1, 800.8321139644669, 334.98116963016344, 406.17631162696944, 463.1526347308606, 445.68643388774973, 443.1198879454823, 499.02154339374204, 471.8213245588483, 423.10655851702586, 401.9259466966568, 427.3793077949641, 390.3263858001731, 409.81108800743334, 371.5705637327504]
slip_r:  371.5705637327504
16
glc__D_e_1:  -0.8262297390762114
glc__D_e_1:  -15.94573550879161
glc__D_e_1:  -19.208321053389202
glc__D_e_1:  -0.1124878441236632
glc__D_e_1:  -31.39793615449497
glc__D_e_1:  -17.009016215171716
glc__D_e_1:  -10.980985251155468
gl

glc__D_e_1:  -1.8913140420068901
glc__D_e_1:  -2.6637914383745436
glc__D_e_1:  -1.0602030159006188
glc__D_e_1:  -0.6334513077791721
glc__D_e_1:  -0.11105066572626265
glc__D_e_1:  -0.2518415474875795
glc__D_e_1:  -0.5052999660409574
glc__D_e_1:  -0.8446428666520533
glc__D_e_1:  -1.8068723419130235
glc__D_e_1:  -1.9093185283500154
glc__D_e_1:  -1.6143683471499854
glc__D_e_1:  -1.8776195543129317
glc__D_e_1:  -3.7194251513567957
glc__D_e_1:  -2.3160810097875566
glc__D_e_1:  -3.6435350200485193
glc__D_e_1:  -5.306179620403807
glc__D_e_1:  -1.563210410109287
glc__D_e_1:  -8.18567610783212
glc__D_e_1:  -4.56674341306593
glc__D_e_1:  -6.306789674994187
glc__D_e_1:  -5.85294988761606
glc__D_e_1:  -6.358061191151968
glc__D_e_1:  -6.105995818491522
glc__D_e_1:  -5.370911972181507
strain_number:  0 4.4
strain_various:  0 3.2516662395352527
strain_number:  1 0.0
strain_various:  1 0.0
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1, 800.8321139644669, 334.98116963016344, 406.17631162696944, 463.15263473

In [11]:
#35
biomass_x_initial, biomass, number, various = time_space_state([model0, model1], ['BIOMASS_Ec_iML1515_core_75p37M', 'BIOMASS_SC5_notrace'], [0.7074, 0.28], [5, 30], [[0.13, 10, 1268], [200, 50, 3410]], public_metabolism, vmax, km, 1/15, 1, 0.1, 30, 0.1, 2.29)
print('The initial distribution of microorganism is: ',biomass_x_initial)
print('The change process of microorganism distribution is: ',biomass)
print('The mean numbers of microorganisms are: ', number)
print('The standard deviations of microorganisms are: ', various)

{0: array([ 1,  7,  2, 15,  4,  9, 17, 14, 16, 11,  5, 11,  1, 10, 20, 10,  2,
        7,  1,  3,  8, 13,  7,  3, 11,  7,  9,  3,  3, 13]), 1: array([11, 20,  1, 16, 10, 18,  7, 20, 19,  4, 18,  1,  4, 18, 20,  5,  6,
        8,  5, 19, 19, 12, 20, 13,  5, 16, 15,  3, 12, 20])}
1
strain_number:  0 9.333333333333334
strain_various:  0 6.2840185302789235
strain_number:  1 12.166666666666666
strain_various:  1 6.573093301905546
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1]
slip_r:  1.1
_1:  {0: array([ 1,  7,  2, 15,  4,  9, 17, 14, 16, 11,  5, 11,  1, 10, 20, 10,  2,
        7,  1,  3,  8, 13,  7,  3, 11,  7,  9,  3,  3, 13]), 1: array([11, 20,  1, 16, 10, 18,  7, 20, 19,  4, 18,  1,  4, 18, 20,  5,  6,
        8,  5, 19, 19, 12, 20, 13,  5, 16, 15,  3, 12, 20])}
_2:  {0: [0.49885752915679404, 0.2857391744693639], 1: [0.49885752915679404, 0.2857391744693639], 2: [0.49885752915679404, 0.2857391744693639], 3: [0.49885752915679404, 0.2857391744693639], 4: [0.49885752915679404, 0.28573917446936

glc__D_e_1:  -5.969753813485834
glc__D_e_1:  -18.921668100181936
glc__D_e_1:  -12.701259652347302
glc__D_e_1:  -14.543970166139903
glc__D_e_1:  -10.543673130988438
glc__D_e_1:  -20.930998884410084
glc__D_e_1:  -7.607496122611068
glc__D_e_1:  -17.514713819010467
glc__D_e_1:  -17.65067661603442
glc__D_e_1:  -13.776501005222382
glc__D_e_1:  -13.853251650226749
strain_number:  0 37.43333333333333
strain_various:  0 15.499856630161311
strain_number:  1 1.5333333333333334
strain_various:  1 1.7838784213679537
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1, 910.6525327742432, 304.9495614035087, 322.7944971694972, 442.34077983420093, 536.1447972462915, 550.9760558956228, 506.9128694635036, 522.7519638370783, 461.0778757555531, 502.9909074219429, 531.0056201909766, 462.5126055000253, 433.89223827181087, 435.5346699215219, 379.4684070250502]
slip_r:  379.4684070250502
17
glc__D_e_1:  -3.874153337358692
glc__D_e_1:  -1.9878093946968773
glc__D_e_1:  -20.93772280267873
glc__D_e_1:  -11.812561641874211
g

glc__D_e_1:  -0.2756602798517783
glc__D_e_1:  -0.47157435959807426
glc__D_e_1:  -0.20371714245236527
glc__D_e_1:  -0.4048213540344505
glc__D_e_1:  -0.5270534181214519
glc__D_e_1:  -3.488083540492145
glc__D_e_1:  -4.910950376048869
glc__D_e_1:  -5.563234100264031
glc__D_e_1:  -5.04564328574434
glc__D_e_1:  -12.069195853332644
glc__D_e_1:  -5.290873631758468
glc__D_e_1:  -6.157721926752511
glc__D_e_1:  -7.414212144750945
glc__D_e_1:  -4.696404508553402
glc__D_e_1:  -3.3520677777015626
glc__D_e_1:  -5.05818043240034
glc__D_e_1:  -4.253653603542053
glc__D_e_1:  -6.258543223691949
glc__D_e_1:  -3.1469669387213894
glc__D_e_1:  -4.376228541851983
strain_number:  0 4.866666666666666
strain_various:  0 3.887872883152889
strain_number:  1 0.0
strain_various:  1 0.0
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1, 910.6525327742432, 304.9495614035087, 322.7944971694972, 442.34077983420093, 536.1447972462915, 550.9760558956228, 506.9128694635036, 522.7519638370783, 461.0778757555531, 502.9909074219429, 

In [12]:
#30
biomass_x_initial, biomass, number, various = time_space_state([model0, model1], ['BIOMASS_Ec_iML1515_core_75p37M', 'BIOMASS_SC5_notrace'], [0.7074, 0.28], [5, 30], [[0.13, 10, 1268], [200, 50, 3410]], public_metabolism, vmax, km, 1/15, 1, 0.1, 30, 0.06, 2.45)
print('The initial distribution of microorganism is: ',biomass_x_initial)
print('The change process of microorganism distribution is: ',biomass)
print('The mean numbers of microorganisms are: ', number)
print('The standard deviations of microorganisms are: ', various)

{0: array([20, 11, 18, 13, 18, 19,  1, 20, 12, 12,  7,  4,  3,  3,  7, 15,  3,
        9,  6,  7, 15,  2,  3,  4, 17,  3,  1, 11,  7,  8]), 1: array([ 1,  7, 20,  2,  8,  6,  5, 12,  2,  4, 12, 17, 17, 12, 14,  7, 18,
        6,  9,  6,  7,  9, 13,  9, 16,  9, 19, 13,  5, 17])}
1
strain_number:  0 10.733333333333333
strain_various:  0 7.3798524526051485
strain_number:  1 10.066666666666666
strain_various:  1 5.328748028904684
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1]
slip_r:  1.1
_1:  {0: array([20, 11, 18, 13, 18, 19,  1, 20, 12, 12,  7,  4,  3,  3,  7, 15,  3,
        9,  6,  7, 15,  2,  3,  4, 17,  3,  1, 11,  7,  8]), 1: array([ 1,  7, 20,  2,  8,  6,  5, 12,  2,  4, 12, 17, 17, 12, 14,  7, 18,
        6,  9,  6,  7,  9, 13,  9, 16,  9, 19, 13,  5, 17])}
_2:  {0: [0.49885752915679404, 0.2857391744693639], 1: [0.49885752915679404, 0.2857391744693639], 2: [0.49885752915679404, 0.2857391744693639], 3: [0.49885752915679404, 0.2857391744693639], 4: [0.49885752915679404, 0.2857391744693

glc__D_e_1:  -4.433523966694817
glc__D_e_1:  -15.194926915294761
glc__D_e_1:  -4.480712741062218
glc__D_e_1:  -15.164778322619231
glc__D_e_1:  -15.874887102568355
glc__D_e_1:  -19.539984970798308
glc__D_e_1:  -22.478773139046098
strain_number:  0 34.96666666666667
strain_various:  0 17.184262826460984
strain_number:  1 1.7666666666666666
strain_various:  1 1.8199511592958264
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1, 743.5332316677129, 408.17874316945523, 385.30989843489846, 436.5370254745255, 457.0742832374024, 453.94338389397814, 436.3519310690669, 387.38602185325334, 421.3054867644683, 410.4349660785921, 411.464899717018, 372.36400479282827, 312.6661426325706, 347.23804762299164]
slip_r:  347.23804762299164
16
glc__D_e_1:  -4.006176622360016
glc__D_e_1:  -9.40586075725916
glc__D_e_1:  -1.9426895797205088
glc__D_e_1:  -5.761240661526705
glc__D_e_1:  -7.245694796343471
glc__D_e_1:  -8.850574511718362
glc__D_e_1:  -11.118715533911557
glc__D_e_1:  -8.618465252063958
glc__D_e_1:  -16.830

glc__D_e_1:  -0.20733756992820562
glc__D_e_1:  -3.4158327274440263
glc__D_e_1:  -2.7022949249859343
glc__D_e_1:  -2.3656151357925457
glc__D_e_1:  -5.5338549137863575
glc__D_e_1:  -3.8138506953897906
glc__D_e_1:  -10.93011584553656
glc__D_e_1:  -4.724928022844731
glc__D_e_1:  -6.678674149616732
glc__D_e_1:  -3.046226960709065
glc__D_e_1:  -4.00844061808351
glc__D_e_1:  -7.507619291537104
glc__D_e_1:  -2.5809608677045084
glc__D_e_1:  -3.14980194990128
glc__D_e_1:  -6.59264464726836
glc__D_e_1:  -7.931642620343432
glc__D_e_1:  -4.3126152940575855
glc__D_e_1:  -5.749954893539162
glc__D_e_1:  -5.8384073836156025
glc__D_e_1:  -2.999011725831582
glc__D_e_1:  -3.1811796862662582
strain_number:  0 5.066666666666666
strain_various:  0 4.178782385123951
strain_number:  1 0.0
strain_various:  1 0.0
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1, 743.5332316677129, 408.17874316945523, 385.30989843489846, 436.5370254745255, 457.0742832374024, 453.94338389397814, 436.3519310690669, 387.38602185325334, 421

In [13]:
#30
biomass_x_initial, biomass, number, various = time_space_state([model0, model1], ['BIOMASS_Ec_iML1515_core_75p37M', 'BIOMASS_SC5_notrace'], [0.7074, 0.28], [5, 30], [[0.13, 10, 1268], [200, 50, 3410]], public_metabolism, vmax, km, 1/15, 1, 0.1, 30, 0.1, 2.45)
print('The initial distribution of microorganism is: ',biomass_x_initial)
print('The change process of microorganism distribution is: ',biomass)
print('The mean numbers of microorganisms are: ', number)
print('The standard deviations of microorganisms are: ', various)

{0: array([ 2, 10,  3,  3, 14,  1,  3, 10, 18, 11, 20, 11,  1,  5,  7,  8, 18,
        5, 15,  8, 14, 20,  6,  8, 10, 13, 19, 14,  8, 19]), 1: array([20,  3,  1,  9, 17, 18, 19, 12, 10, 13,  1, 12,  3,  1, 20,  3, 11,
       15, 15,  6, 20, 18, 15, 19, 19,  6, 16, 13,  8,  4])}
1
strain_number:  0 11.766666666666667
strain_various:  0 7.017517763109372
strain_number:  1 11.566666666666666
strain_various:  1 6.448169421540419
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1]
slip_r:  1.1
_1:  {0: array([ 2, 10,  3,  3, 14,  1,  3, 10, 18, 11, 20, 11,  1,  5,  7,  8, 18,
        5, 15,  8, 14, 20,  6,  8, 10, 13, 19, 14,  8, 19]), 1: array([20,  3,  1,  9, 17, 18, 19, 12, 10, 13,  1, 12,  3,  1, 20,  3, 11,
       15, 15,  6, 20, 18, 15, 19, 19,  6, 16, 13,  8,  4])}
_2:  {0: [0.49885752915679404, 0.2857391744693639], 1: [0.49885752915679404, 0.2857391744693639], 2: [0.49885752915679404, 0.2857391744693639], 3: [0.49885752915679404, 0.2857391744693639], 4: [0.49885752915679404, 0.28573917446936

glc__D_e_1:  -17.42007644244071
glc__D_e_1:  -16.042789095024194
glc__D_e_1:  -0.6563233584524042
glc__D_e_1:  -0.5129436967743428
glc__D_e_1:  -5.451535213972693
glc__D_e_1:  -26.320156599573203
glc__D_e_1:  -2.965960071817122
glc__D_e_1:  -0.3403649607276257
glc__D_e_1:  -8.921383462840309
glc__D_e_1:  -13.19347173836526
glc__D_e_1:  -1.516373474487219
glc__D_e_1:  -0.9238861102565146
glc__D_e_1:  -8.46421118810796
glc__D_e_1:  -30.117972530438596
glc__D_e_1:  -3.723082578421277
glc__D_e_1:  -0.311683737695866
glc__D_e_1:  -5.250275254894216
glc__D_e_1:  -23.01980931610822
glc__D_e_1:  -0.03161001888333459
glc__D_e_1:  -5.205983225772401
glc__D_e_1:  -21.445472412092723
glc__D_e_1:  -12.364968549189735
strain_number:  0 34.8
strain_various:  0 19.925528014751997
strain_number:  1 1.4666666666666666
strain_various:  1 1.2578641509408806
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1, 749.5761481165893, 358.27071612597933, 387.6489113956219, 441.91084706057615, 458.353149444597, 451.1123785

glc__D_e_1:  -4.824753523548742
glc__D_e_1:  -2.4915669997775347
glc__D_e_1:  -1.7896323014927977
glc__D_e_1:  -3.6471491123535316
glc__D_e_1:  -1.5945174803756377
glc__D_e_1:  -2.0393992609502423
glc__D_e_1:  -0.8785170232618276
glc__D_e_1:  -0.2537529993569779
glc__D_e_1:  -0.297235127290583
glc__D_e_1:  -0.8551680143513326
glc__D_e_1:  -0.24967620305880678
glc__D_e_1:  -0.15218709723783008
strain_number:  0 11.233333333333333
strain_various:  0 21.414455138734887
strain_number:  1 0.03333333333333333
strain_various:  1 0.1795054935711501
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1, 749.5761481165893, 358.27071612597933, 387.6489113956219, 441.91084706057615, 458.353149444597, 451.11237851296, 437.408659040866, 459.7463626746552, 414.3081869973315, 289.7376744784483, 289.2551419209942, 400.94028671694343, 418.47834403396655, 387.08022791665803, 299.17973828203986, 219.88706340399224, 219.50457818116612, 212.8817619578489, 195.4569155547739]
slip_r:  195.4569155547739
21
glc__D_e_1:  -2

strain_number:  0 0.06666666666666667
strain_various:  0 0.24944382578492943
strain_number:  1 0.0
strain_various:  1 0.0
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1, 749.5761481165893, 358.27071612597933, 387.6489113956219, 441.91084706057615, 458.353149444597, 451.11237851296, 437.408659040866, 459.7463626746552, 414.3081869973315, 289.7376744784483, 289.2551419209942, 400.94028671694343, 418.47834403396655, 387.08022791665803, 299.17973828203986, 219.88706340399224, 219.50457818116612, 212.8817619578489, 195.4569155547739, 211.83899008953207, 139.48981341548478, 160.40618259143054, 134.11654135338347, 41.32352941176471, 44.17857142857143, 49.16666666666667, 23.75, 22.5]
slip_r:  22.5
30
strain_number:  0 0.03333333333333333
strain_various:  0 0.1795054935711501
strain_number:  1 0.0
strain_various:  1 0.0
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1, 749.5761481165893, 358.27071612597933, 387.6489113956219, 441.91084706057615, 458.353149444597, 451.11237851296, 437.408659040866, 459.7463

In [14]:
biomass_x_initial, biomass, number, various = time_space_state([model0, model1], ['BIOMASS_Ec_iML1515_core_75p37M', 'BIOMASS_SC5_notrace'], [0.7074, 0.28], [5, 30], [[0.13, 10, 1268], [200, 50, 3410]], public_metabolism, vmax, km, 1/15, 1, 0.1, 30, 0.06, 2.65)
print('The initial distribution of microorganism is: ',biomass_x_initial)
print('The change process of microorganism distribution is: ',biomass)
print('The mean numbers of microorganisms are: ', number)
print('The standard deviations of microorganisms are: ', various)

{0: array([13,  2,  5, 17,  2,  1,  1,  6,  4,  9, 16,  6, 14,  8, 20,  2, 12,
       18,  2,  9,  6,  4, 10, 19,  3, 15,  4, 17,  8,  9]), 1: array([ 2, 10,  1, 14,  6, 14,  4, 12, 14, 15,  7, 15, 18, 15,  6, 17,  5,
        8, 20, 14, 15,  5, 14,  4, 16, 12, 17, 19,  1, 19])}
1
strain_number:  0 10.033333333333333
strain_various:  0 7.087469380690278
strain_number:  1 11.3
strain_various:  1 5.745432968889291
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1]
slip_r:  1.1
_1:  {0: array([13,  2,  5, 17,  2,  1,  1,  6,  4,  9, 16,  6, 14,  8, 20,  2, 12,
       18,  2,  9,  6,  4, 10, 19,  3, 15,  4, 17,  8,  9]), 1: array([ 2, 10,  1, 14,  6, 14,  4, 12, 14, 15,  7, 15, 18, 15,  6, 17,  5,
        8, 20, 14, 15,  5, 14,  4, 16, 12, 17, 19,  1, 19])}
_2:  {0: [0.49885752915679404, 0.2857391744693639], 1: [0.49885752915679404, 0.2857391744693639], 2: [0.49885752915679404, 0.2857391744693639], 3: [0.49885752915679404, 0.2857391744693639], 4: [0.49885752915679404, 0.2857391744693639], 5: [0.498

glc__D_e_1:  -1.17414154384128
glc__D_e_1:  -19.035806096837295
glc__D_e_1:  -18.255795531076075
glc__D_e_1:  -1.0642780820791586
glc__D_e_1:  -20.572577538934844
glc__D_e_1:  -30.090514816068293
glc__D_e_1:  -3.026950815618463
glc__D_e_1:  -0.3750893337943646
glc__D_e_1:  -20.40373550278067
glc__D_e_1:  -20.43915476403715
glc__D_e_1:  -1.828996987938524
glc__D_e_1:  -22.377786941643024
glc__D_e_1:  -14.802378117848226
glc__D_e_1:  -0.2675850449745454
glc__D_e_1:  -15.324787652337621
glc__D_e_1:  -20.132295594124447
glc__D_e_1:  -1.522137818025822
glc__D_e_1:  -1.2554392725356285
glc__D_e_1:  -17.6416584566076
glc__D_e_1:  -17.413821018449887
glc__D_e_1:  -2.513030715045012
glc__D_e_1:  -1.1439573678155215
glc__D_e_1:  -12.3267094305813
glc__D_e_1:  -14.190986671117287
glc__D_e_1:  -1.6622157435411378
glc__D_e_1:  -3.0890301658154407
glc__D_e_1:  -16.873515789234315
strain_number:  0 32.5
strain_various:  0 12.194944307649243
strain_number:  1 1.6333333333333333
strain_various:  1 1.47

glc__D_e_1:  -14.972293122016257
glc__D_e_1:  -24.96566020017484
glc__D_e_1:  -17.980498995700238
glc__D_e_1:  -30.71613084462993
glc__D_e_1:  -34.74633458589157
glc__D_e_1:  -38.63822359516059
glc__D_e_1:  -21.205407306651637
glc__D_e_1:  -6.228383178695193
glc__D_e_1:  -14.057669613219613
glc__D_e_1:  -1.9311781688074494
glc__D_e_1:  -0.31702510740276724
glc__D_e_1:  -0.07424144976505131
glc__D_e_1:  -0.8717562185036111
glc__D_e_1:  -0.8724424733518104
glc__D_e_1:  -0.05881254621645371
glc__D_e_1:  -1.0664633262030823
glc__D_e_1:  -0.04179560262408066
glc__D_e_1:  -1.1309250633081942
glc__D_e_1:  -0.8189472126764749
glc__D_e_1:  -0.11369679527737153
glc__D_e_1:  -0.6207897709122612
glc__D_e_1:  -0.04712028139124286
glc__D_e_1:  -0.6703743808034642
strain_number:  0 9.9
strain_various:  0 14.695464152814866
strain_number:  1 0.0
strain_various:  1 0.0
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1, 926.91769111951, 330.97528129764976, 387.7837162837163, 433.8655368754054, 395.9235928545139

strain_number:  0 0.0
strain_various:  0 0.0
strain_number:  1 0.0
strain_various:  1 0.0
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1, 926.91769111951, 330.97528129764976, 387.7837162837163, 433.8655368754054, 395.9235928545139, 370.54311061745267, 384.8610158870558, 443.30006660325375, 409.1061821235652, 466.69198025203826, 540.1198869059497, 469.5425322759334, 343.187946200296, 344.8855172444079, 368.4823820752555, 340.49680189225205, 305.15224905669777, 237.42612216424945, 174.5747555902018, 243.44038282196465, 234.9371212121212, 180.6562187812188, 104.54953560371517, 79.55987394957984, 67.75, 49.5, 102.5, 45.0, 15.0]
slip_r:  15.0
31
strain_number:  0 0.0
strain_various:  0 0.0
strain_number:  1 0.0
strain_various:  1 0.0
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1, 926.91769111951, 330.97528129764976, 387.7837162837163, 433.8655368754054, 395.9235928545139, 370.54311061745267, 384.8610158870558, 443.30006660325375, 409.1061821235652, 466.69198025203826, 540.1198869059497, 469.54253227

In [15]:
biomass_x_initial, biomass, number, various = time_space_state([model0, model1], ['BIOMASS_Ec_iML1515_core_75p37M', 'BIOMASS_SC5_notrace'], [0.7074, 0.28], [5, 30], [[0.13, 10, 1268], [200, 50, 3410]], public_metabolism, vmax, km, 1/15, 1, 0.1, 30, 0.1, 2.65)
print('The initial distribution of microorganism is: ',biomass_x_initial)
print('The change process of microorganism distribution is: ',biomass)
print('The mean numbers of microorganisms are: ', number)
print('The standard deviations of microorganisms are: ', various)

{0: array([16, 16, 16, 19, 12,  9,  2, 16,  4,  8,  6, 17, 19, 10, 20,  8,  3,
        9,  5,  1,  8,  1,  9, 15, 16, 12, 11, 12,  5, 18]), 1: array([13,  9, 10, 20,  5, 18, 11,  6,  6,  4,  6,  4,  5, 16, 18,  6, 17,
       17, 20, 16, 12,  2,  1,  6, 19, 11, 17, 16, 16, 19])}
1
strain_number:  0 12.533333333333333
strain_various:  0 6.825117011613956
strain_number:  1 11.533333333333333
strain_various:  1 5.981824322246703
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1]
slip_r:  1.1
_1:  {0: array([16, 16, 16, 19, 12,  9,  2, 16,  4,  8,  6, 17, 19, 10, 20,  8,  3,
        9,  5,  1,  8,  1,  9, 15, 16, 12, 11, 12,  5, 18]), 1: array([13,  9, 10, 20,  5, 18, 11,  6,  6,  4,  6,  4,  5, 16, 18,  6, 17,
       17, 20, 16, 12,  2,  1,  6, 19, 11, 17, 16, 16, 19])}
_2:  {0: [0.49885752915679404, 0.2857391744693639], 1: [0.49885752915679404, 0.2857391744693639], 2: [0.49885752915679404, 0.2857391744693639], 3: [0.49885752915679404, 0.2857391744693639], 4: [0.49885752915679404, 0.28573917446936

glc__D_e_1:  -21.283459239931982
glc__D_e_1:  -16.766564033079685
glc__D_e_1:  -0.04186527652907657
glc__D_e_1:  -1.5065046829872415
glc__D_e_1:  -16.50234280980446
glc__D_e_1:  -1.1158770732326708
glc__D_e_1:  -0.1687444699846128
glc__D_e_1:  -10.831149820619775
glc__D_e_1:  -19.186759103053788
glc__D_e_1:  -2.096948039085781
glc__D_e_1:  -1.6792786016768648
glc__D_e_1:  -12.862030664442644
glc__D_e_1:  -19.96708377417257
glc__D_e_1:  -2.0256000465064545
glc__D_e_1:  -0.6657019911007679
glc__D_e_1:  -6.644986932560357
strain_number:  0 41.46666666666667
strain_various:  0 21.685222823132086
strain_number:  1 1.3
strain_various:  1 1.6155494421403513
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1, 590.4017684456924, 406.129216951779, 388.57469504180034, 431.96059130927546, 431.00896481614365, 485.03540445003114, 525.4597273230838, 416.0854143976822, 416.41731054609625, 426.2368872380104, 398.4576712252302, 416.4181680958075, 389.763799830486]
slip_r:  389.763799830486
15
glc__D_e_1:  -4.816

glc__D_e_1:  -0.7597991158991859
glc__D_e_1:  -0.12353115489127808
glc__D_e_1:  -0.2771031617821468
glc__D_e_1:  -0.9970937539810316
glc__D_e_1:  -2.5577953025073024
glc__D_e_1:  -3.0736448447720175
glc__D_e_1:  -2.8266104870781494
glc__D_e_1:  -2.242216128830544
glc__D_e_1:  -2.8669786064356866
glc__D_e_1:  -3.5811350944794658
glc__D_e_1:  -1.5827838138212569
glc__D_e_1:  -1.8056138008987244
glc__D_e_1:  -4.613683313482419
glc__D_e_1:  -3.0667790032462
glc__D_e_1:  -11.934100050813685
glc__D_e_1:  -10.437499624028803
glc__D_e_1:  -4.464591133140564
glc__D_e_1:  -3.3524689109651042
glc__D_e_1:  -4.857838230610884
glc__D_e_1:  -1.4260774109368017
glc__D_e_1:  -2.086515036589783
strain_number:  0 3.9
strain_various:  0 4.02781992315115
strain_number:  1 0.0
strain_various:  1 0.0
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1, 590.4017684456924, 406.129216951779, 388.57469504180034, 431.96059130927546, 431.00896481614365, 485.03540445003114, 525.4597273230838, 416.0854143976822, 416.417310546